# Get-RscMssqlInstance
Typically, when dealing with SQL Server, you start with an Instance. This will hold true with Rubrik as well, as an Instance is the home to a database. If you search for a database in Rubrik, you could get back multiple entries for that database because there are multiple copies on different Instances. By getting the SQL Server Instance Id, you can use that later to ensure that when you search for a database, you get the database specific to an Instance. Additionally, the Instance ID will also be used for other operations like Exporting a Databaase, Live Mount, setting up Log Shipping. 

# Workflow
To get information about the SQL Server Instance, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the host id that the SQL Server is installed on. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [1]:
Set-Location '/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20231108-58     v20231127-20



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host Information

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [4]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster
$RscMssqlInstance

DEBUG: -Running Get-RscMssqlInstance
DEBUG: -  Creating Host Query
DEBUG: [New_RscQueryMssql] RscGqlPSCmdlet.cs:141:GetOp()
  Operation from -Operation: TopLevelDescendants
DEBUG: [New_RscQueryMssql] RscGqlPSCmdlet.cs:141:GetOp()
  Operation from -Operation: TopLevelDescendants
DEBUG: [New_RscQueryMssql] RscDynParamPSCmdlet.cs:162:GetDynamicParameters()
    returning
    AddField: Name=AddField Type=System.String[] IsSet=False Value=
    RemoveField: Name=RemoveField Type=System.String[] IsSet=False Value=
  
DEBUG: [New_RscQueryMssql] RscBasePSCmdlet.cs:154:BeginProcessing()
  Cmdlet Inputs:
    Operation: "TopLevelDescendants"
    FieldProfile: [RubrikSecurityCloud.Exploration+Profile]DEFAULT
DEBUG: [New_RscQueryMssql] RscGqlPSCmdlet.cs:326:GetOperationsDir()
  GetOperationsDir()
DEBUG: [New_RscQueryMssql] RscGqlPSCmdlet.cs:223:ProcessRecord()
  op dir: /Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sd

### Get the Instance ID
All of the infomation about the host is returned. Part of the returned object is the information about the instance. It can be found under `PhysicalChildConnection.Nodes`

In [ ]:
$RscMssqlInstance.PhysicalChildConnection.Nodes

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc